# 类型系统 

+ 与二进制存储结构相关联的成为元类型/原语类型： `primitive type`, 如`Int32`, `Int64`这种，在定义元类型时要声明**位数**， 且位数必须为8的倍数
+ 类型名第一个字母大写，包括自定义的类型、结构体等

## 元类型

In [11]:
# 256位正数
primitive type Int256 <: Integer 256 end

In [12]:
# 8位浮点数
primitive type Float8 <: AbstractFloat 8 end

与抽象概念相关，被用来继承的类型为`Abstract Type`, 如`Real`, `Integer`, `AbstractFloat`

## 抽象类型

In [13]:
abstract type name end
abstract type family_name <: name end

In [14]:
family_name <: name

true

In [15]:
supertype(family_name)

name

In [16]:
subtypes(name)

1-element Vector{Any}:
 family_name

In [17]:
# 并不会递归查询
subtypes(Integer)

4-element Vector{Any}:
 Bool
 Int256
 Signed
 Unsigned

##  类型的类型

In [18]:
typeof(Int64)

DataType

In [19]:
typeof(DataType)

DataType

所有类型的类型是`DataType`，所有类型的父类型是`Any`, 所有类型的子类型都包含`Union`

In [20]:
Real <: Any, typeof(Union) <: DataType

(true, true)

In [21]:
# DataType和Any都是类型，Any是所有类型的父类型，而DataType是所有类型的类型
DataType <: Any, typeof(Any) <: DataType

(true, true)

In [22]:
supertype(Union)

Type{T}

`DataType`的父类型是参数化类型`Type{T}`

In [23]:
supertype(DataType)

Type{T}

`Type{T}`可以看作类型选择器

In [45]:
isa(Int64, Type{Int64})

true

In [47]:
# Type{T}的使用
function convert_type(::Type{T}, x::Integer) where T <: AbstractFloat
    return convert(T, x)
end

convert_type (generic function with 1 method)

In [49]:
convert_type(Float64, 1)

1.0000

##  复合类型

In [50]:
abstract type Foo end

In [51]:
struct FooA <: Foo
    a::Any
    b::Float64
end

In [52]:
dump(FooA)

FooA <: Foo
  a::Any
  b::Float64


In [54]:
FooA <: Foo, fieldnames(FooA)

(true, (:a, :b))

###  构造函数

+ 按照struct中类型指定，手动指定每一个field的类型
+ 不指定每一个field的类型，julia会自动进行convert到struct的类型指定（不建议）

> 一个struct实例化后，如果更改某一个field的值，他会自动将该值convert到struct的类型指定。

任何一个struct在定义时对field进行了类型声明，那么任何一个实例化的struct的field的值都是该field的类型

In [56]:
fooa = FooA("xue", 23)

FooA("xue", 23.0000)

In [57]:
dump(fooa)

FooA
  a: String "xue"
  b: Float64 23.0000


## Callable Struct(闭包、closure)

函数内定义函数(好像也是闭包)， Julia很难进行优化
```Julia
funtion f1()
    function f2()
    end
end
```

In [42]:
struct abb
    a::Int64
    b::Float64
end

In [57]:
# 目的是利用构建的struct创建一个新函数
function (f::abb)(c)
    return f.a + f.b + c
end

In [49]:
abb_f = abb(1, 2)

abb(1, 2.0000)

In [56]:
methods(abb_f)

# 1 method for callable object:
 [1] (f::abb)(c)
     @ In[46]:2

In [53]:
abb_f(1)

4.0000

## 参数化类型

### 参数化复合类型

In [58]:
mutable struct Point1D{T <: Real}
    x::T
end

In [69]:
# Poin1D可以被看作一种简写
Point1D{<:Real} == Point1D

true

In [75]:
# 即使参数存在继承关系, 参数化符合类型也不存在继承关系
# 专业术语叫作“抗变”
# 抗变只发生在container类型中，标量没有这个特性
Point1D{Int64} <: Point1D{Integer}

false

Julia自带的数据类型像是`Dict`，`Vector`，`Matrix`也可以看成结构体一样的东西（container），因此也遵循抗变的特征

在Julia定义函数时（详见[函数参数类型标注](https://docs.julialang.org/en/v1/manual/functions/#Argument-type-declarations)），使用`：：`对参数进行类型声明
+ 如`f(x::Real) = x + 1`，那么`x`只要是`Real`的任意子类型，该函数都能被调用。
+ 如果参数是以container的形式出现，务必要注意抗变的特性。
    + 例如函数`f(x::AbstractVector{Real})`，输入只要是AbstractVector的子类型就可以，但是这个container内的元素类型必须也是Real， 因为`AbstractVector{Float64}`并不是`AbstractVector{Real}`的子类型。最好的办法是声明`x::AbstractVector{<:Real}`或者`f(x::AbstractVector{T}) where {T <: Real}`
    + 如果是更加复杂的嵌套container，更要注意抗变的特性，尤其是和参数类型结合在一起的时候。例如`f(x::Dict{Int, Vector{<:Real}})`，当输入`Dict(1 => [1, 2.0], 2 => [1, 3.0])`时找不到函数去执行，原因在于抗变的特性，虽然container内的元素是子类型，但是container不是子类型

In [88]:
function f1(x::Dict{Int, Vector{<:Real}})
    rst = 0.0
    for (k, v) in x
        rst += sum(v)
    end
    return rst
end

f1 (generic function with 1 method)

In [89]:
f1(Dict(1 => [1, 2.0], 2 => [1, 3.0]))

LoadError: MethodError: no method matching f1(::Dict{Int64, Vector{Float64}})

[0mClosest candidates are:
[0m  f1([91m::Dict{Int64, Vector{<:Real}}[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[88]:1[24m[39m


In [90]:
function f2(x::Dict{Int, <:Vector{<:Real}})
    rst = 0.0
    for (k, v) in x
        rst += sum(v)
    end
    return rst
end

f2 (generic function with 1 method)

In [91]:
@which f2(Dict(1 => [1, 2.0], 2 => [1, 3.0]))

f2(x::Dict{Int64, <:Vector{<:Real}})
     @ Main In[90]:1

In [95]:
typeof(Dict(1 => [1, 2.0], 2 => [1, 3.0])) <: Dict{Int64, <:Vector{<:Real}}

true

In [101]:
Dict{Int, T} == Dict{Int64, <:Vector{<:Real}}

LoadError: UndefVarError: `T` not defined

### 参数化元类型

In [11]:
primitive type ptr{T} 64 end

### 参数化基本原理（跳过）

从这一块到最后有点麻烦，先跳过

### 参数化继承(跳过）

## 常用数据类型

### 元组

In [1]:
tp1 = (1.0, 2.0, 3.0, 4.0)

(1.0000, 2.0000, 3.0000, 4.0000)

In [2]:
typeof(tp1)

NTuple{4, Float64}

In [3]:
# NTuple不是一个复合类型
tp3 = NTuple{3, Float64}(1.0, 2.0, 3.0)

LoadError: MethodError: no method matching Tuple{Float64, Float64, Float64}(::Float64, ::Float64, ::Float64)

[0mClosest candidates are:
[0m  (::Type{T})(::Any) where T<:Tuple
[0m[90m   @[39m [90mBase[39m [90m[4mtuple.jl:391[24m[39m


In [4]:
tp1[1]

1.0000

In [5]:
tp2 = (1, 2, (3, 4, 5)..., 6, 7)

(1, 2, 3, 4, 5, 6, 7)

In [6]:
tp3 = (1, "a", "china")

(1, "a", "china")

### 命名元组

In [7]:
ntp1 = (a = 1, b = 2, c = 3)

(a = 1, b = 2, c = 3)

In [8]:
typeof(ntp1)

@NamedTuple{a::Int64, b::Int64, c::Int64}

In [9]:
ntp1.a

1

In [10]:
# 说明命名元组是一个参数化复合类型
ntp2 = NamedTuple{(:a, :b, :c),Tuple{Int64,Int64,Float64}}((1, 2, 3.0))

(a = 1, b = 2, c = 3.0000)

### 键值对

In [11]:
dump(Pair) # 先只关注body部分吧，其它的部分先别管

UnionAll
  var: TypeVar
    name: Symbol A
    lb: Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol B
      lb: Union{}
      ub: Any
    body: Pair{A, B} <: Any
      first::A
      second::B


In [12]:
fieldnames(Pair)

(:first, :second)

In [13]:
p1 = Pair(1, 3.2)

1 => 3.2000

In [14]:
typeof(p1)

Pair{Int64, Float64}

In [15]:
p2 = 2 => "3"

2 => "3"

In [16]:
p1.first

1

In [17]:
p2.second

"3"

In [18]:
# Pair to Turple
Tuple(p1)

(1, 3.2000)

### 字典

In [19]:
d1 = Dict(1 => 1.1, 2 => 2.2)

Dict{Int64, Float64} with 2 entries:
  2 => 2.2000
  1 => 1.1000

In [20]:
d2 = Dict{String,Int64}("a" => 3, "b" => 4.0)

Dict{String, Int64} with 2 entries:
  "b" => 4
  "a" => 3

In [21]:
# 字典推导式
d3 = Dict(i => i^2 for i ∈ 1:10)

Dict{Int64, Int64} with 10 entries:
  5  => 25
  4  => 16
  6  => 36
  7  => 49
  2  => 4
  10 => 100
  9  => 81
  8  => 64
  3  => 9
  1  => 1

In [22]:
ntp2

(a = 1, b = 2, c = 3.0000)

In [23]:
pairs(ntp2)

pairs(::NamedTuple) with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0000

In [24]:
# NamedTuple to Dict
d4 = Dict(pairs(ntp2))

Dict{Symbol, Real} with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0000

In [25]:
d4[:b]

2

In [26]:
delete!(d4, :a)

Dict{Symbol, Real} with 2 entries:
  :b => 2
  :c => 3.0000

In [27]:
pop!(d4, :b)

2

In [28]:
d4

Dict{Symbol, Real} with 1 entry:
  :c => 3.0000

In [29]:
d4.count

1

In [30]:
keys(d2)

KeySet for a Dict{String, Int64} with 2 entries. Keys:
  "b"
  "a"

In [31]:
for key in keys(d2)
    println("$key \t", d2[key])
end

b 	4
a 	3


In [32]:
values(d2)

ValueIterator for a Dict{String, Int64} with 2 entries. Values:
  4
  3

In [33]:
# collect iterator
collect(values(d2))

2-element Vector{Int64}:
 4
 3

In [34]:
# Set(可迭代数据结构)
a = Set(1:5)

Set{Int64} with 5 elements:
  5
  4
  2
  3
  1

In [35]:
b = Set([1, 2, 3])

Set{Int64} with 3 elements:
  2
  3
  1

In [36]:
b = Set(("a", 1, 2))

Set{Any} with 3 elements:
  2
  "a"
  1

In [37]:
1 in b

true

In [38]:
isempty(a)

false

In [39]:
empty!(a)

Set{Int64}()

### 缺失值missing 

In [40]:
# NaN是一个数并不代表缺失值

In [41]:
missing + 1

missing

In [42]:
ismissing([1, 2, 3, missing])

false

In [43]:
ismissing.([1, 2, 3, missing])

4-element BitVector:
 0
 0
 0
 1

In [44]:
isless(missing, Inf)

false

### nothing不需要在内存中表达

## 其它数据类型

Julia有一个`DataStructures.jl`， 这个包内置了很多Julia本身没有的数据结构

### `OrderedDict`

In [65]:
using DataStructures

In [66]:
# 有序字典
d1 = OrderedDict('a' => 1, 'b' => 2, 'c' => 3)

OrderedDict{Char, Int64} with 3 entries:
  'a' => 1
  'b' => 2
  'c' => 3

In [67]:
# 无序字典（Julia内置）
d2 = Dict('a' => 1, 'b' => 2, 'c' => 3)

Dict{Char, Int64} with 3 entries:
  'a' => 1
  'c' => 3
  'b' => 2

In [68]:
?methodswith

search: methodswith



```
methodswith(typ[, module or function]; supertypes::Bool=false])
```

Return an array of methods with an argument of type `typ`.

The optional second argument restricts the search to a particular module or function (the default is all top-level modules).

If keyword `supertypes` is `true`, also return arguments with a parent type of `typ`, excluding type `Any`.


`methodswith`函数可以查看所有以某个数据结构为参数的所有函数， 例如`methodswith(OrderedDict)`可以查看所有适用于`OrderedDict`的函数， 可以看出包的作者给`OrderedDict`适配了很多的方法

In [112]:
methodswith(OrderedDict)

[1] copy(d::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:55
[2] delete!(h::OrderedDict, key) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:446
[3] empty(d::OrderedDict{K, V}) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:84
[4] empty(d::OrderedDict, ::Type{K}, ::Type{V}) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:85
[5] empty!(h::OrderedDict{K, V}) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:225
[6] get(default::Union{Function, Type}, h::OrderedDict{K, V}, key) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:393
[7] get(h::OrderedDict{K, V}, key, default) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:388
[8] get!(default::Union{Function, Type}, h::OrderedDict{K, V}, key0) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:359
[9] get!(h::OrderedDict{K, V}, key0, default) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:344
[10] getindex(h::OrderedDict{K, V}, key) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:383
[11] getkey(h::OrderedDict{K, V}, key, default) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:401
[12] haskey(h::OrderedDict, key) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:398
[13] isempty(d::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:88
[14] iterate(t::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:452
[15] iterate(t::OrderedDict, i) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:457
[16] last(h::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:511
[17] length(d::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:87
[18] merge(d::OrderedDict, others::AbstractDict...) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:486
[19] merge(combine::Function, d::OrderedDict, others::AbstractDict...) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:491
[20] pop!(h::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:412
[21] pop!(h::OrderedDict, key) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:426
[22] pop!(h::OrderedDict, key, default) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:431
[23] popfirst!(h::OrderedDict) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:419
[24] setindex!(h::OrderedDict{K, V}, v0, key0) where {K, V} @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:325
[25] similar(d::OrderedDict) @ OrderedCollections deprecated.jl:103
[26] sizehint!(d::OrderedDict, newsz) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\ordered_dict.jl:211
[27] sort!(d::OrderedDict; byvalue, args...) @ OrderedCollections C:\Users\xue\.julia\packages\OrderedCollections\9C4Uz\src\dict_sorting.jl:4

In [113]:
zipit = zip(1:5, 'a':'e')

zip(1:5, 'a':1:'e')

In [114]:
OrderedDict(zipit)

OrderedDict{Int64, Char} with 5 entries:
  1 => 'a'
  2 => 'b'
  3 => 'c'
  4 => 'd'
  5 => 'e'